In [1]:
!pip install geemap

In [2]:
import hydrofunctions as hf
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import folium
import geemap.foliumap as geemap
import ee
import matplotlib.dates as mdates
import hydrofns

%matplotlib inline

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWilHccs_PUkE9pxBXGfB5VF2g2VwB5Wq-p3t5v2nakJaqnGWk7bLUw

Successfully saved authorization token.


In [4]:
# site number for NWIS gage (Pine Bluff AR Site)
pineBluff = ['07263650']

# Coordinates for the bounds of a rectangle at Pine Bluff AR
pineBluff_xMin = -92.4
pineBluff_yMin = 34
pineBluff_xMax = -91.7
pineBluff_yMax = 34.4

# Ignore wet and dry season labels - this notebook for testing only, and I'm too lazy to change all variable names.
pineBluff_wet_season_start = '2019-01-01'
pineBluff_wet_season_end = '2019-01-30'

pineBluff_dry_season_start = '2019-06-01'
pineBluff_dry_season_end = '2019-06-30'

# pineBluff_Map = geemap.Map(center=[34.2, -92], zoom=11)
# pineBluff_Map

In [19]:
# Import the Sentinel 2 collection as sentinel 2
sentinel2 = ee.ImageCollection(
    'COPERNICUS/S2_SR').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Pine Bluff geometry - NEED TO FIGURE OUT HOW TO CHANGE BOUNDS GEOMETRY TO CAPTURE ONLY RIVER AND BANKS!!!
# using ee.geometry.polygon with four/five sets of coordinates didn't work for me.
pineBluff_rectangleGeoJSON = ee.Geometry.Rectangle(
    [
        [pineBluff_xMin, pineBluff_yMin],
        [pineBluff_xMax, pineBluff_yMax]
    ]
)

# Palettes for Visualization
trueColor_palette = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
waterPalette = ['red', 'yellow', 'green', 'blue']

In [20]:
# Import the Sentinel 2 collection as sentinel 2

pineBluff_bnds = ee.FeatureCollection([
    ee.Feature(
        pineBluff_rectangleGeoJSON,
        {'name': 'Pine Bluff, AR', 'fill': 1}),
])

pineBluff_dry_season = sentinel2.filterBounds(pineBluff_bnds).filterDate(
    pineBluff_dry_season_start, pineBluff_dry_season_end).mosaic().clip(pineBluff_bnds)

pineBluff_wet_season = sentinel2.filterBounds(pineBluff_bnds).filterDate(
    pineBluff_wet_season_start, pineBluff_wet_season_end).mosaic().clip(pineBluff_bnds)

In [21]:
Map = geemap.Map(center=[34.2, -92], zoom=11)
Map.addLayer(pineBluff_dry_season, trueColor_palette,
             '2019 Dry season true color')
Map.addLayerControl()
Map

In [22]:
Map = geemap.Map(center=[34.2, -92], zoom=11)
Map.addLayer(pineBluff_wet_season, trueColor_palette,
             '2019 Wet season true color')
Map.addLayerControl()
Map

In [23]:
pineBluff_dry_season_Map = geemap.Map(center=[34.2, -92], zoom=11)
pineBluff_ndwi_dry = pineBluff_dry_season.normalizedDifference(['B3', 'B8'])
pineBluff_dry_season_Map.addLayer(pineBluff_ndwi_dry, {
                                 'min': -1, 'max': 0.5, 'palette': waterPalette}, '2019 Dry season NDWI')
pineBluff_dry_season_Map

In [24]:
pineBluff_wet_season_Map = geemap.Map(center=[34.2, -92], zoom=11)
pineBluff_ndwi_wet = pineBluff_wet_season.normalizedDifference(['B3', 'B8'])
pineBluff_wet_season_Map.addLayer(pineBluff_ndwi_wet, {
                                 'min': -1, 'max': 0.5, 'palette': waterPalette}, '2019 Wet season NDWI')
pineBluff_wet_season_Map

In [25]:
pineBluff_wet_season_Map = geemap.Map(center=[34.2, -92], zoom=11)

ndwi_threshold_ar = 0
pineBluff_wet_season_image = pineBluff_ndwi_wet.gt(
    ndwi_threshold_ar).selfMask()

pineBluff_wet_season_Map.addLayer(pineBluff_wet_season_image, {
                                 'palette': 'blue'}, 'Water image')
pineBluff_wet_season_Map

In [26]:
pineBluff_dry_season_Map = geemap.Map(center=[34.2, -92], zoom=11)

ndwi_threshold_ar = 0
pineBluff_dry_season_image = pineBluff_ndwi_dry.gt(
    ndwi_threshold_ar).selfMask()

pineBluff_dry_season_Map.addLayer(pineBluff_dry_season_image, {
                                 'palette': 'blue'}, 'Water image')
pineBluff_dry_season_Map

In [ ]:
# Reduce the region. The region parameter is the Feature geometry.
pineBluff_count_pixels_wet = pineBluff_wet_season_image.reduceRegion(**{
    'reducer': ee.Reducer.count(),
    'geometry': pineBluff_bnds.geometry(),
    'scale': 10,
    'maxPixels': 1e9
})

# Reduce the region. The region parameter is the Feature geometry.
pineBluff_count_pixels_dry = pineBluff_dry_season_image.reduceRegion(**{
    'reducer': ee.Reducer.count(),
    'geometry': pineBluff_bnds.geometry(),
    'scale': 10,
    'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
print("Pine Bluff water pixel count in the wet season: ",
      pineBluff_count_pixels_wet.getInfo())
print("Pine Bluff water pixel count in the dry season: ",
      pineBluff_count_pixels_dry.getInfo())

In [ ]:
# pineBluff_site_df = hydrofns.create_df_gageht(
#     pineBluff, pineBluff_wet_season_start, pineBluff_dry_season_end)
# pineBluff_site_df

# # Define plot space, overall size and title
# fig, (ax1) = plt.subplots(1, 1, figsize=(15, 10))

# fig.suptitle(
#     "Stream Gage Height Data for Pine Bluff AR Gage", fontsize=22)

# # Loop for plotting Subplot 1
# pineBluff_site_df.plot(y='gage ht',
#                        title="Daily Gage Ht for Pine Bluff AR Gage (2019)",
#                        label='ARKANSAS RIVER AT PINE BLUFF, AR',
#                        linewidth=3.0,
#                        ax=ax1,
#                        alpha=.8)

# # Define the date format for Subplot 1, set axis labels
# date_form = DateFormatter("%b-%d")
# ax1.xaxis.set_major_formatter(date_form)
# ax1.xaxis.set_major_locator(mdates.WeekdayLocator(interval=5))
# ax1.set(xlabel="Date (Month-Day)",
#         ylabel="Daily Mean Gage Height \n (ft)")

# #plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', prop={'size': 11})